<a href="https://colab.research.google.com/github/AnaKarenDRIV/FinanzasUniversitarias/blob/main/RedesConvusionales/Clasificaci%C3%B3n_de_d%C3%ADgitos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# -------------------------
#  Importaciones
# -------------------------
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, recall_score
import seaborn as sns
from PIL import Image

# -------------------------
# 1) Cargar y explorar el dataset
# -------------------------
# Cargamos el dataset "digits" (8x8 píxeles, 10 clases: dígitos 0-9)
digits = load_digits()

# Las claves del diccionario nos indican qué contiene el dataset
print(digits.keys())
# 'data'  : arreglos 2D plano (n_samples, 64)
# 'images': arreglos 3D con forma (n_samples, 8, 8)
# 'target' : etiquetas enteras

# Imprimir descripción breve del dataset
print(digits.DESCR[:400], "...\n")  # muestra las primeras 400 columnas de la descripción

# Visualizar una imagen de ejemplo
index = 0
image = digits.images[index]
label = digits.target[index]
print('Matriz de la imagen (8x8):\n', image)
plt.figure(figsize=(3, 3))
plt.imshow(image, cmap=plt.cm.gray)
plt.title(f'Dígito: {label}')
plt.axis('off')
plt.show()

# -------------------------
# 2) División en entrenamiento y prueba + one-hot encode
# -------------------------
# Los modelos Keras para clasificación multiclase a menudo esperan etiquetas en formato one-hot
X_train, X_test, y_train, y_test = train_test_split(
    digits.data, digits.target, test_size=0.2, random_state=42
)

# Convertir etiquetas a one-hot (10 clases)
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

print('Forma X_train:', X_train.shape)
print('Forma X_test:', X_test.shape)

# Mostrar cómo se ve el primer elemento antes de escalado (al aplanarlo a 8x8)
reshaped_tensor = tf.reshape(X_train[0], shape=(8, 8))
print('Primer elemento (reshape 8x8) antes de escalado:\n', reshaped_tensor.numpy())

# -------------------------
# 3) Escalado / normalización
# -------------------------
# Elegimos StandardScaler para centrar cada característica en 0 con varianza 1.
# Importante: usamos el mismo scaler (fit sobre X_train) luego para transformar nuevas imágenes.
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Ver cómo se ve el primer elemento tras la normalización (redondeado para mostrar)
reshaped_tensor = tf.reshape(X_train_scaled[0], shape=(8, 8))
reshaped_tensor = tf.floor(reshaped_tensor * 100) / 100
print('Primer elemento (reshape 8x8) tras StandardScaler (redondeado):\n', reshaped_tensor.numpy())

# Para la CNN necesitamos la forma (n_samples, 8, 8, 1)
X_train_cnn = X_train_scaled.reshape((X_train_scaled.shape[0], 8, 8, 1)).astype('float32')
X_test_cnn = X_test_scaled.reshape((X_test_scaled.shape[0], 8, 8, 1)).astype('float32')

# -------------------------
# 4) Definición del modelo CNN
# -------------------------
# Nota: las imágenes son pequeñas (8x8) por lo que la arquitectura debe ser sencilla para no sobreajustar.
model = Sequential([
    Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(8, 8, 1)),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')  # salida con softmax -> probabilidades sobre 10 clases
])

# Mostrar resumen de la arquitectura
model.summary()

# -------------------------
# 5) Compilación
# -------------------------
# Usamos Adam y entropía cruzada categórica (one-hot)
learning_rate = 0.001
adam_optimizer = Adam(learning_rate=learning_rate)
model.compile(optimizer=adam_optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# -------------------------
# 6) Entrenamiento
# -------------------------
# Entrenamos el modelo; validation_split separa parte del X_train para validación interna.
history = model.fit(X_train_cnn, y_train, epochs=10, batch_size=32, validation_split=0.2)

# -------------------------
# 7) Visualizar la curva de pérdida
# -------------------------
plt.plot(history.history['loss'], label='Pérdida de entrenamiento')
plt.plot(history.history['val_loss'], label='Pérdida de validación')
plt.xlabel('Épocas')
plt.ylabel('Pérdida')
plt.legend()
plt.title('Función de pérdida durante el entrenamiento')
plt.show()

# -------------------------
# 8) Evaluación en conjunto de prueba
# -------------------------
loss, accuracy = model.evaluate(X_test_cnn, y_test)
print(f'Loss: {loss:.4f}, Accuracy: {accuracy:.4f}')

# -------------------------
# 9) Matriz de confusión y sensibilidad (recall) por clase
# -------------------------
# Obtener predicciones y convertir de probabilidades a clases
y_pred = model.predict(X_test_cnn)
# clases predichas (0..9)
y_pred_classes = np.argmax(y_pred, axis=1)
# convertir y_test (one-hot) a etiquetas enteras
y_test_classes = np.argmax(y_test, axis=1)

conf_matrix = confusion_matrix(y_test_classes, y_pred_classes)
sensitivity = recall_score(y_test_classes, y_pred_classes, average=None)

# Visualizar matriz de confusión
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()

# Mostrar sensibilidad por clase
print('Sensitivity (Recall) for each class:')
for i in range(10):
    print(f'Class {i}: {sensitivity[i]:.4f}')

# -------------------------
# 10) BONUS: Predecir una imagen externa (mi_numero.png)
# -------------------------
# Para predecir una imagen externa debemos aplicar exactamente el mismo preprocesamiento usado
# en X_train: convertir a escala de grises, redimensionar a 8x8, reescalar los valores al mismo rango
# y aplicar el mismo scaler (fit solo sobre X_train).

# Ejemplo de lectura y preprocesamiento:
ruta = 'mi_numero.png'  # modificar según ubicación real
i = Image.open(ruta).convert('L')  # L -> escala de grises

# Redimensionar a 8x8 (no mantener proporción si queremos exactamente 8x8)
i_resized = i.resize((8, 8), Image.Resampling.LANCZOS)

# Convertir a array y escalar para coincidir con los valores originales (si se desea 0-16 como en la demo):
# Si el dataset original estuviera en 0-16, habría que transformar accordingly. En nuestro pipeline usamos
# StandardScaler sobre los datos planos (ya extraídos con digits.data), así que debemos aplicar la misma
# transformación estadística: primero aplanar, luego aplicar scaler.transform, y dar forma (1,8,8,1).

img_array = np.array(i_resized).astype('float32')
# Si los datos originales estaban en 0-16 (dataset 'digits' lo presentaba así visualmente), se puede mapear.
# Pero dado que usamos StandardScaler fit sobre X_train (en escala original), aplicamos la misma transformación:
img_flat = img_array.reshape(1, -1)  # forma (1, 64)
# IMPORTANTE: usamos el mismo 'scaler' definido y fit sobre X_train
img_scaled = scaler.transform(img_flat)  # (1, 64)
img_cnn = img_scaled.reshape((1, 8, 8, 1)).astype('float32')

# Predecir
pred = model.predict(img_cnn)
digit = np.argmax(pred, axis=1)[0]
print('Predicción:', digit)

# Fin del script. Guardar modelos, usar callbacks (EarlyStopping, ModelCheckpoint) o aumentar data augmentation
# pueden mejorar rendimiento en datasets pequeños o cuando hay sobreajuste.
